# Evolver Loop 3 Analysis

## Critical Issue: Both submissions failed with 'Overlapping trees in group 008'

We need to:
1. Understand the EXACT overlap detection the LB uses
2. Investigate group 008 specifically
3. Find a truly valid submission

In [1]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union
import warnings
warnings.filterwarnings('ignore')

print('Libraries loaded')

Libraries loaded


In [2]:
# Tree geometry constants
TRUNK_W = 0.15
TRUNK_H = 0.2
BASE_W = 0.7
MID_W = 0.4
TOP_W = 0.25
TIP_Y = 0.8
TIER_1_Y = 0.5
TIER_2_Y = 0.25
BASE_Y = 0.0
TRUNK_BOTTOM_Y = -TRUNK_H

BASE_TREE_VERTICES = np.array([
    [0.0, TIP_Y],
    [TOP_W/2, TIER_1_Y],
    [TOP_W/4, TIER_1_Y],
    [MID_W/2, TIER_2_Y],
    [MID_W/4, TIER_2_Y],
    [BASE_W/2, BASE_Y],
    [TRUNK_W/2, BASE_Y],
    [TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, TRUNK_BOTTOM_Y],
    [-TRUNK_W/2, BASE_Y],
    [-BASE_W/2, BASE_Y],
    [-MID_W/4, TIER_2_Y],
    [-MID_W/2, TIER_2_Y],
    [-TOP_W/4, TIER_1_Y],
    [-TOP_W/2, TIER_1_Y],
])

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    rotation_matrix = np.array([[cos_a, -sin_a], [sin_a, cos_a]])
    rotated = BASE_TREE_VERTICES @ rotation_matrix.T
    translated = rotated + np.array([x, y])
    return Polygon(translated)

def parse_submission(df):
    result = df.copy()
    for col in ['x', 'y', 'deg']:
        result[col] = result[col].str.replace('s', '').astype(float)
    return result

print('Helper functions defined')

Helper functions defined


In [3]:
# LB-style overlap check: intersects() and not touches()
def check_lb_overlaps(polygons):
    """Check for overlaps using LB's method: intersects() and not touches()"""
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                overlaps.append((i, j))
    return overlaps

# Also check with area-based method for comparison
def check_area_overlaps(polygons):
    """Check for overlaps using area > 0 method"""
    overlaps = []
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            intersection = polygons[i].intersection(polygons[j])
            if intersection.area > 0:
                overlaps.append((i, j, intersection.area))
    return overlaps

print('Overlap check functions defined')

Overlap check functions defined


In [4]:
# Load the current submission and check group 008 specifically
print('Loading current submission...')
current_df = pd.read_csv('/home/submission/submission.csv')
current_parsed = parse_submission(current_df)
print(f'Shape: {current_parsed.shape}')

# Check group 008
prefix = '008_'
config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
print(f'\nGroup 008 has {len(config_df)} trees')
print(config_df)

Loading current submission...
Shape: (20100, 4)

Group 008 has 8 trees
       id         x         y         deg
28  008_0 -0.249562 -0.411092   51.766738
29  008_1  0.664668 -0.857305  113.629378
30  008_2 -0.664668  0.257305  293.629378
31  008_3  0.557305  0.326837  203.629378
32  008_4  0.249562 -0.188908  231.766738
33  008_5 -0.206589 -0.855262  293.629378
34  008_6  0.206589  0.255262  113.629378
35  008_7 -0.557305 -0.926837   23.629378


In [5]:
# Create polygons for group 008 and check overlaps
polygons_008 = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]

lb_overlaps = check_lb_overlaps(polygons_008)
area_overlaps = check_area_overlaps(polygons_008)

print(f'LB-style overlaps in group 008: {len(lb_overlaps)}')
if lb_overlaps:
    print(f'Overlapping pairs: {lb_overlaps}')

print(f'\nArea-based overlaps in group 008: {len(area_overlaps)}')
if area_overlaps:
    for i, j, area in area_overlaps:
        print(f'  Trees {i} and {j}: area = {area}')

LB-style overlaps in group 008: 0

Area-based overlaps in group 008: 0


In [6]:
# Check ALL groups with LB-style overlap detection
print('Checking ALL groups with LB-style overlap detection...')

lb_overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = current_parsed[current_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        print(f'WARNING: Group {n} has {len(config_df)} trees instead of {n}')
        continue
    
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    if overlaps:
        lb_overlap_groups.append((n, len(overlaps)))

print(f'\nGroups with LB-style overlaps: {len(lb_overlap_groups)}')
if lb_overlap_groups:
    print('First 20 groups with overlaps:')
    for n, count in lb_overlap_groups[:20]:
        print(f'  Group {n:03d}: {count} overlapping pairs')

Checking ALL groups with LB-style overlap detection...



Groups with LB-style overlaps: 0


In [7]:
# Check chistyakov_best.csv - the evaluator says this has 0 overlaps
print('\nChecking chistyakov_best.csv...')
chistyakov_df = pd.read_csv('/home/code/preoptimized/chistyakov_best.csv')
chistyakov_parsed = parse_submission(chistyakov_df)

# Check group 008 in chistyakov
prefix = '008_'
config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
overlaps = check_lb_overlaps(polygons)
print(f'Group 008 in chistyakov_best.csv: {len(overlaps)} LB-style overlaps')

# Check ALL groups
chistyakov_overlap_groups = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    config_df = chistyakov_parsed[chistyakov_parsed['id'].str.startswith(prefix)]
    if len(config_df) != n:
        continue
    polygons = [create_tree_polygon(row['x'], row['y'], row['deg']) for _, row in config_df.iterrows()]
    overlaps = check_lb_overlaps(polygons)
    if overlaps:
        chistyakov_overlap_groups.append((n, len(overlaps)))

print(f'Total groups with LB-style overlaps in chistyakov_best.csv: {len(chistyakov_overlap_groups)}')
if chistyakov_overlap_groups:
    print(f'Groups: {chistyakov_overlap_groups}')


Checking chistyakov_best.csv...
Group 008 in chistyakov_best.csv: 0 LB-style overlaps


Total groups with LB-style overlaps in chistyakov_best.csv: 0


In [ ]:
# The LB is rejecting with "Overlapping trees in group 008" but our check shows 0 overlaps
# This could mean:
# 1. The LB uses different tree geometry
# 2. The LB uses different precision
# 3. There's a floating point issue in how we read the CSV

# Let's check the raw CSV values for group 008
print('Raw CSV values for group 008:')
raw_df = pd.read_csv('/home/submission/submission.csv')
group_008 = raw_df[raw_df['id'].str.startswith('008_')]
print(group_008)

# Check if there are any precision issues
print('\nParsed values:')
for _, row in group_008.iterrows():
    x = float(row['x'].replace('s', ''))
    y = float(row['y'].replace('s', ''))
    deg = float(row['deg'].replace('s', ''))
    print(f"{row['id']}: x={x:.20f}, y={y:.20f}, deg={deg:.20f}")